# "[Code] 네이버 금융 api를 이용한 금융데이터 크롤링. "

- toc:true
- branch: master
- badges: true
- comments: true
- author: Daniel Lee - edypidy
- categories: [code, 크롤링, api, 네이버, 네이버 금융]

아주 흔하디 흔한 네이버 금융 api를 이용한 금융 데이터 크롤링을 해보겠습니다.

In [4]:
import requests
import json
import pandas as pd

In [11]:
corp_code = '000020'
startTime = 20160104
endTime = 20210730
timeframe = 'day'
url = f'https://api.finance.naver.com/\
siseJson.naver?symbol={corp_code}&\
requestType=1&\
startTime={startTime}&\
endTime={endTime}&\
timeframe={timeframe}'

result = requests.post(url)

In [12]:
url

'https://api.finance.naver.com/siseJson.naver?symbol=000020&requestType=1&startTime=20160104&endTime=20210730&timeframe=day'

위의 url은 아래와 같습니다. 참고로 \\는 줄바꿈을 위해 쓴 것이니 

        f'https://api.finance.naver.com/siseJson.naver?symbol={corp_code}&requestType=1&startTime={startTime}&endTime={endTime}&timeframe={timeframe}'

와 같이 써도 됩니다.

In [16]:
result

<Response [200]>

response로 200을 받은 것을 보니 잘 받아온 것 같습니다.

In [18]:
result.text[:500]

'\n [[\'날짜\', \'시가\', \'고가\', \'저가\', \'종가\', \'거래량\', \'외국인소진율\'],\n\n\t\n\t\n\t\t\n["20160104", 8130, 8150, 7920, 8140, 281440, 7.45],\n\t\t\n["20160105", 8040, 8250, 8000, 8190, 243179, 7.49],\n\t\t\n["20160106", 8200, 8590, 8110, 8550, 609906, 7.63],\n\t\t\n["20160107", 8470, 8690, 8190, 8380, 704752, 7.59],\n\t\t\n["20160108", 8210, 8900, 8130, 8770, 802330, 7.6],\n\t\t\n["20160111", 8870, 10100, 8800, 9380, 2844188, 7.25],\n\t\t\n["20160112", 9560, 9560, 8930, 9020, 834633, 7.05],\n\t\t\n["20160113", 9190, 9200, 8600, 8780, 885059, 6.89],\n\t\t'

result의 text 속성엔 위와 같이 요청한 데이터가 담겨서 옵니다.

replace 메소드와 strip메소드를 사용해서 아래와 같이 데이터를 정제해줍니다.

In [28]:
result.text.replace("'",'"').strip()[:500]

'[["날짜", "시가", "고가", "저가", "종가", "거래량", "외국인소진율"],\n\n\t\n\t\n\t\t\n["20160104", 8130, 8150, 7920, 8140, 281440, 7.45],\n\t\t\n["20160105", 8040, 8250, 8000, 8190, 243179, 7.49],\n\t\t\n["20160106", 8200, 8590, 8110, 8550, 609906, 7.63],\n\t\t\n["20160107", 8470, 8690, 8190, 8380, 704752, 7.59],\n\t\t\n["20160108", 8210, 8900, 8130, 8770, 802330, 7.6],\n\t\t\n["20160111", 8870, 10100, 8800, 9380, 2844188, 7.25],\n\t\t\n["20160112", 9560, 9560, 8930, 9020, 834633, 7.05],\n\t\t\n["20160113", 9190, 9200, 8600, 8780, 885059, 6.89],\n\t\t\n['

json의 loads 메소드를 사용해서 데이터 형태를 아래와 같이 바꿔줍니다.

In [25]:
json.loads(result.text.replace("'",'"').strip())[:5]

[['날짜', '시가', '고가', '저가', '종가', '거래량', '외국인소진율'],
 ['20160104', 8130, 8150, 7920, 8140, 281440, 7.45],
 ['20160105', 8040, 8250, 8000, 8190, 243179, 7.49],
 ['20160106', 8200, 8590, 8110, 8550, 609906, 7.63],
 ['20160107', 8470, 8690, 8190, 8380, 704752, 7.59]]

여기서부턴 그냥 pandas 데이터 프레임 만드는 것과 동일합니다. 정리하면 코드는 아래와 같습니다.

In [26]:
import requests
import json
import pandas as pd

corp_code = '000020'
startTime = 20160104
endTime = 20210730
timeframe = 'day'
url = f'https://api.finance.naver.com/\
siseJson.naver?symbol={corp_code}&\
requestType=1&\
startTime={startTime}&\
endTime={endTime}&\
timeframe={timeframe}'

result = requests.post(url)

data = result.text.replace("'",'"').strip()
data = json.loads(data)
data = pd.DataFrame(data[1:], columns=data[0])
data['날짜'] = pd.to_datetime(data['날짜'])
data['종목코드'] = corp_code

In [27]:
data

,날짜,시가,고가,저가,종가,거래량,외국인소진율,종목코드
0,2016-01-04,8130,8150,7920,8140,281440,7.45,000020
1,2016-01-05,8040,8250,8000,8190,243179,7.49,000020
2,2016-01-06,8200,8590,8110,8550,609906,7.63,000020
3,2016-01-07,8470,8690,8190,8380,704752,7.59,000020
4,2016-01-08,8210,8900,8130,8770,802330,7.60,000020
...,...,...,...,...,...,...,...,...
1367,2021-07-26,14700,14700,14350,14400,192180,2.54,000020
1368,2021-07-27,14500,14550,14350,14400,96069,2.55,000020
1369,2021-07-28,14300,14600,14300,14450,94912,2.56,000020
1370,2021-07-29,14500,14650,14500,14600,83468,2.56,000020


셀레니움 라이브러리를 이용하는 것이 아니라 api를 이용해 크롤링을 한다면 네이버 금융 api 이외에도 데이터 api가 공개되어있는 사이트들을 크롤링 할 수 있습니다.